In [0]:
%run ./00_functions_and_libraries

/databricks/spark/python/pyspark/sql/pandas/functions.py:386: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
 warnings.warn(

In [0]:
%run ./01_params

Out[7]: 'if params[\'Database\'] == "prod":\n params[\'RF_LOC\'] = "X"\nelif params[\'Database\'] == "impl":\n params[\'Database\']\nelif params[\'Database\']=="val":\n params[\'Database\']\nelif params[\'db\']=="dev1":\n params[\'Database\']\nelif params[\'Database\'] == "dev0":\n params[\'RF_LOC\']\nelse:\n raise ValueError(\'{} is invalid or unmapped\'.format(db))'

In [0]:
spark.conf.set('spark.sql.execution.arrow.enabled', True)
spark.conf.set('spark.sql.execution.arrow.fallback.enabled', False)
spark.conf.set("spark.sql.session.timeZone", "America/New_York")
spark.conf.set(
  params["AzureSASLocation"],
  dbutils.secrets.get(scope=params["AzureSASScope"],key=params["AzureSASKey"])
)


db = params["Database"]
checkpoint = params["sparkCheckpointDir_claims"]
cfgfile = params['ClaimConfigFile']
intDataDir = params["intermediateDataDir_claims"]
tempDataDir = params["tempdir_claims"]
outboundDir = None #params["AzureET3Mount"]+"prod/outbound"
archiveDir = None #params["AzureET3Mount"]+"prod/archive/outbound"
snowflakeDir = None #params["AzureET3Mount"]+"prod/snowflake/"

spark.sql(f"create database if not exists {db}")
spark.sql(f"use {db}")
spark.sparkContext.setCheckpointDir(checkpoint)
print(f"Input Parameters:\n   Database: {db}\n   Spark Checkpoint Dir: {checkpoint}\n   Weights Config File:{cfgfile}\n   Intermediate Dir:{intDataDir}\n   Temp Dir:{tempDataDir}\n   Outbound Dir:{outboundDir}\n   Archive Dir:{archiveDir}\n   Snowflake Dir:{snowflakeDir}")
assert "ml" in params['sparkVersion'], "Spark ML runtime is a requirement!"
assert int(params['sparkVersion'].split(".")[0]) >= 8, "Spark version 8 or above is a requirement!"

Input Parameters:
 Database: prod
 Spark Checkpoint Dir: /mnt/edfr/et3/prod/HelperData_claims/checkpoint
 Weights Config File:wasbs://et3@edfrvaproddbrstoragegen2.blob.core.usgovcloudapi.net/prod/HelperData/Claim_Batch_Dedup_config_sample_data.json
 Intermediate Dir:/mnt/edfr/et3/prod/HelperData_claims
 Temp Dir:/mnt/edfr/et3/prod/HelperData_claims/temp
 Outbound Dir:None
 Archive Dir:None
 Snowflake Dir:None

In [0]:
w_LName = params["WEIGHT: Last Name"]
w_FName = params["WEIGHT: First Name"]
w_MName = params["WEIGHT: Middle Initial/Name"]
w_Street = params["WEIGHT: Patient's Home Address"]
w_City = params["WEIGHT: Patient's Home City"]
w_County = params["WEIGHT: Patient's Home County"]
w_State = params["WEIGHT: Patient's Home State"]
w_Zip = params["WEIGHT: Patient's Home ZIP Code"]
w_SSN = params["WEIGHT: Social Security Number"]
w_Gender = params["WEIGHT: Gender"]
w_Race = params["WEIGHT: Race"]
w_Age = params["WEIGHT: Age"]
w_Dob = params["WEIGHT: Date of Birth"]
w_Lic = params["WEIGHT: Driver's License Number"]
w_MBI = params["WEIGHT: MBI"]
w_StateDL = params["WEIGHT: State Issuing Driver's License"]

w_total_active = 0
for x in [w_LName, w_FName, w_MName,w_Street, w_City, w_County, w_State, w_Zip,w_Gender, w_Race, w_Dob, w_Age, w_Lic, w_StateDL, w_SSN,w_MBI]:
  w_total_active=+float(x)

match_threshold = params["Match Threshold"]

In [0]:
#######################
"""PARSE CONFIG FILE"""
#######################
config = spark.read.option("multiline", True).json(cfgfile)

# "Mappings" tell us how to take data from the source columns and map them to our logical contexts for matching
#    e.g. concat the f_name and l_name columns from source1 and map that to PersonNames
#           and take full_name from source2 and map that to PersonNames.
mappings = (config
            .select(explode("contextConfig").alias("contextConfig"))
            .withColumn("contextName", col("contextConfig.contextName"))
            .withColumn("sourceMappings", explode("contextConfig.sourceMappings"))
            .withColumn("deltaTable", col("sourceMappings.deltaTable"))
            .withColumn("sourceCols", col("sourceMappings.sourceCols"))
            .withColumn("targetAlias", col("sourceMappings.targetAlias"))
            .drop("contextConfig", "sourceMappings")
           )

# "Sources" are the delta tables containing the source data to be matched
#   This configuration will tell us:
#        - what the existing primary key is
#        - whether to look for duplicates within the source (i.e. selfDedup = True/False)
sources = (config
           .select(explode("sourceConfig").alias("sourceConfig"))
           .withColumn("deltaTable", col("sourceConfig.deltaTable"))
           .withColumn("primaryKey", col("sourceConfig.primaryKey"))
           .withColumn("selfDedup", col("sourceConfig.selfDedup"))
           .drop("sourceConfig")
          )

# "Tokenizers" are methods for matching.  We can tokenize by spaces/words, or by ngram.
#    This config is set per matching context.  So, if you want to match on PersonNames and BusinessNames, there should be an entry for both
#    We also need to know how to set up the MinHash algorithm for each context
tokenizers = (config
              .select(explode("contextConfig").alias("contextConfig"))
              .withColumn("contextName", col("contextConfig.contextName"))
              .withColumn("tokenizerConfig", explode("contextConfig.tokenizerConfig"))
              .withColumn("mode", col("tokenizerConfig.mode"))
              .withColumn("jaccardIndexThreshold", col("tokenizerConfig.binningConfig.jaccardIndexThreshold"))
              .withColumn("numHashTables", col("tokenizerConfig.binningConfig.numHashTables"))
              .withColumn("idfCutoff", col("tokenizerConfig.binningConfig.idfCutoff"))
              .withColumn("termFreq", col("tokenizerConfig.tf"))
              .drop("contextConfig", "tokenizerConfig")
             )

sources.show(5)
tokenizers.show(20)
mappings.show(100, False)

+----------+-------------+---------+
deltaTable| primaryKey|selfDedup|
+----------+-------------+---------+
 pcr_meta| [mp_id]| false|
 claims_pm|[CLM_UNIQ_ID]| false|
+----------+-------------+---------+

+--------------+-------+---------------------+-------------+---------+--------+
 contextName| mode|jaccardIndexThreshold|numHashTables|idfCutoff|termFreq|
+--------------+-------+---------------------+-------------+---------+--------+
 PersonName|default| 0.3| 4| 2.0| null|
 PersonName|trigram| null| null| null| null|
 FirstName|trigram| null| null| null| null|
 LastName|trigram| null| null| null| null|
 SSN|default| 0.05| 4| 0.01| null|
 Address|default| 0.3| 4| 5.1| null|
Address_street|trigram| null| null| null| null|
 Address_city|trigram| null| null| null| null|
 Address_state|trigram| null| null| null| false|
 Address_zip|default| null| null| null| null|
 Gender|default| null| null| null| false|
 DOB|default| null| null| null| null|
 Race|default| null| null| null| false|
 MiddleName|trigram| null| null| null| null|
Address_county|trigram| null| null| null| null|
 Age|numeric| null| null| null| null|
 MBI|default| null| null| null| null|
+--------------+-------+---------------------+-------------+---------+--------+

+--------------+----------+------------------------------------+--------------+
contextName |deltaTable|sourceCols |targetAlias |
+--------------+----------+------------------------------------+--------------+
PersonName |pcr_meta |[first_name, last_name] |null |
PersonName |claims_pm |[BENE_1ST_NAME, BENE_LAST_NAME] |null |
FirstName |pcr_meta |[first_name] |null |
FirstName |claims_pm |[BENE_1ST_NAME] |null |
LastName |pcr_meta |[last_name] |null |
LastName |claims_pm |[BENE_LAST_NAME] |null |
SSN |pcr_meta |[ssn] |null |
SSN |claims_pm |[BENE_SSN_NUM] |null |
Address |pcr_meta |[address, city, zip_code] |primary |
Address |claims_pm |[BENE_LINE_1_ADR, City, SRC_ZIP5_CD]|primary |
Address_street|pcr_meta |[address] |primary_street|
Address_street|claims_pm |[BENE_LINE_1_ADR] |primary_street|
Address_city |pcr_meta |[city] |null |
Address_city |claims_pm |[City] |null |
Address_state |pcr_meta |[state] |null |
Address_state |claims_pm |[SRC_USPS_STATE_CD] |null |
Address_zip |pcr_meta |[zip_code] |null |
Address_zip |claims_pm |[SRC_ZIP5_CD] |null |
Gender |pcr_meta |[gender] |null |
Gender |claims_pm |[BENE_SEX_CD] |null |
DOB |pcr_meta |[dob] |null |
DOB |claims_pm |[bene_BRTH_DT] |null |
Race |pcr_meta |[race] |null |
Race |claims_pm |[BENE_RACE_CD] |null |
MiddleName |pcr_meta |[MIDDLE_NAME] |null |
MiddleName |claims_pm |[BENE_MIDL_NAME] |null |
Address_county|pcr_meta |[county] |null |
Address_county|claims_pm |[County] |null |
Age |pcr_meta |[AGE] |null |
Age |claims_pm |[Age] |null |
MBI |pcr_meta |[MBI] |null |
MBI |claims_pm |[BENE_MBI_ID] |null |
+--------------+----------+------------------------------------+--------------+

In [0]:
#########################
"""CONFIGURING SOURCES"""
#########################
sourceList = []  # Each element of this list will be a Dataframe of a source Delta table
sourceCount = len(sources.select("deltaTable").collect())  # Loop once for each source Delta table in config file

# Lets get a list of all of the columns needed from each delta table
# If a column isn't listed in the mappings, then we don't explicitly need it.
allColumns = (mappings
              .withColumn("sourceCols", explode("sourceCols"))
              .groupBy("deltaTable")
              .agg(collect_set("sourceCols").alias("columns"))
             )

print("\nReading source delta tables into dataframes...")
for sNum in range(sourceCount):
  # There will be 1 iteration of this loop per source table. 
  # sName/sKey/sCol ==> Name/PrimaryKey/ColumnsNamesAsString
  sName = allColumns.select("deltaTable").rdd.collect()[sNum][0]
  sKey = sources.filter(col("deltaTable")==sName).select("primaryKey").collect()[0][0]
  sCol = ", ".join(sKey + allColumns.select("columns").rdd.collect()[sNum][0])
  source = spark.sql(f"select {sCol} from {sName} where er_flag = 1").fillna("")
  assert source.select(sKey[0]).distinct().count() == source.count(), f"Error: Specified primaryKey \"{sKey[0]}\" for source \"{sName}\" is not unique"
  source = source.withColumn(sKey[0], concat_ws("__", *[lit(sName), col(sKey[0])])) # sKey must be a single column
  source = source.withColumnRenamed(sKey[0], "sourceId")
  source = normCols(source)
  sourceList.append(source)
  print(f"   Source '{sName}' is ready")

Reading source delta tables into dataframes...
 Source 'pcr_meta' is ready
 Source 'claims_pm' is ready

In [0]:
##########################
"""CONFIGURING CONTEXTS"""
##########################

# There will be 1 entry per context, keyed on the context name in the following two dictionaries
# The dictionary values will be dataframes containing the context values and the source names & record ids they originated from
contextDict = {}     # Dataframes in this list are deduped so there is one row per value in each context.
contextDictRaw = {}  # Dataframes in this list are not deduped
# Q: Why dedup in the step above?
# A: If we have 1000 records that all contain the value "some cool words" and 50 records with the value "some cooler words", 
#    we shouldn't compare "some cool words" to "some cooler words" 50,000 times. Instead, we just do it once and keep track of all
#    the other records that share the same value

contextNames = mappings.select("contextName").distinct()
contextCount = contextNames.count()  # We will be looping over all of the context in the contextConfig section of the config file
contextColumns = (mappings  # For each context, let's only keep the source columns that we need.
                  .withColumn("sourceCols", explode("sourceCols"))
                  .groupBy("contextName", "deltaTable")
                  .agg(collect_set("sourceCols").alias("columns"))
                 )

# We are doing a lot of up-front deduplication so as to minimize the work done during MinHashLSH. 
# We need a method for undoing the deduplication afterward - 
#    e.g. if 2 records are identical, keep one and MinHash it, then for every candidate pair it generates, 
#         create a second candidate pair using the record we dropped earlier
# So, we store the full list of ALL records and what values they contained
# We'll join back on this table after doing the MinHash approach.
schema = StructType([
  StructField("sourceId", StringType()), 
  StructField("featureId", StringType()), 
  StructField("valueId", StringType()),
  StructField("contextName", StringType())
])
source_to_feature = spark.createDataFrame([], schema)

# 1 loop per context
for cNum in range(contextCount):
  cName = contextNames.collect()[cNum][0]
  thisContext = []
  thisContextRaw = []
  for sNum in range(sourceCount): # 1 inner loop per source delta table
    sName = allColumns.select("deltaTable").rdd.collect()[sNum][0]   # Get the delta table name
    cCol = (contextColumns
            .filter((col("contextName")==cName) & (col("deltaTable")==sName))
            .select("columns")
           ).collect()[0][0]  # Get the list of the required columns
    _context = sourceList[sNum].select(["sourceId"] + cCol)  # keep only the required columns
    _context = map_to_targets(_context, mappings, cName, sName)  # map the source columns to our target columns
    _context = melt(_context, ["sourceId"], _context.columns[1:])  # pivot the data so we get 1 row per context value
    # Note, in the above step, this pivot is necessary, because 1 source could have multiple representations of the same context
    #    e.g. full_name, known_alias, also_known_as, etc.  OR home_address, work_address, alt_address, etc.
    #    So 1 source row could produce more than 1 row in our context table... 1 row per context value.
    
    _context = drop_bad_values(add_hashes(_context)).withColumn("contextName", lit(cName))
    thisContextRaw.append(_context)
    thisContext.append(dedup_values(_context).checkpoint())  # Here we dedup on the values to minimize work during MinHash
    # Note, in the above step we are checkpointing here because otherwise we higt an NPE at the tokenize() step
    
  combinedContext = reduce(DataFrame.unionAll, thisContext)  # Combine this contexts data from all of the sources
  combinedContextRaw = reduce(DataFrame.unionAll, thisContextRaw)  # Same thing as above but for non-deduped data
  contextDict[cName] = combinedContext
  contextDictRaw[cName] = combinedContextRaw
  combinedContextRaw.write.format("delta").mode("overwrite").save(f"{intDataDir}/delta/SILVER/context_{cName}")
  combinedContextRaw = spark.read.format("delta").load(f"{intDataDir}/delta/SILVER/context_{cName}")
  
  # This is our lookup table to get back to the original record granularity
  source_to_feature = source_to_feature.union(combinedContextRaw.select("sourceId", "featureId", "valueId", "contextName"))

source_to_feature.createOrReplaceTempView("source_to_feature")

for k,v in contextDict.items():
  print("context info")
  print(f"  name: {k}")
  print(f"  record count: {v.count()}")
  print(". example: ")
  try:
    pprint.pprint(v.take(1)[0].asDict())
  except IndexError:
    print(Exception(f"\n Warning: The Context Table {k} is empty. It will not contribute to probabilistic matching."))
  print("\n")

In [0]:
#######################
"""CONFIGURING PAIRS"""
#######################

# featuresList is a list of dictionaries; 1 dict per tokenization strategy
#    each dictionary will contain the contextName, the tokenizer method, and a mapping of sourceIds to vectorized/weighted features
featuresList = []

# pairsList is a list of Dataframes containing the candidate pairs generated from each tokenization/binning strategy
#    each dataframe in this list contains just 2 columns: id1, id2 (where these ids each identify a unique value for the context)
#      e.g. The DF may say that featureId 55 forms a candidate pair with featureId 145.  
#           We can go back to the source_to_feature table to see that featureId 55 == "some cool words" and featureId 145 == "some cooler words"
pairsList = []

ip_schema = StructType([StructField("sourceId1", StringType()), StructField("sourceId2", StringType())])
ident_pairs = spark.createDataFrame([], ip_schema)

for TCFG in tokenizers.collect():
  # iterating over each tokenization strategy.  
  #   TCFG has the following schemao
  #      element 0: contextName (string)
  #      element 1: mode (string)
  #      element 2: jaccardIndexThreshold (double)
  #      element 3: numHashTables (long)
  #      element 4: idfCutoff (double)
  #      element 5: termFreq (boolean)
  _termFreq = True if TCFG[5] is None else False
  
  # Here we finally tokenize our data so, for instance, "some cool words" becomes ["some", "cool", "words"]
  # Note we are dropping rows where the tokens column is empty
  _tokens = tokenize(contextDict.get(TCFG[0]), mode=TCFG[1], sid="sourceId").filter(size(col("tokens"))>0)
  if _tokens.count() == 0:
    continue
  
  # Here we convert the token arrays into sparse vectors with IDF weights per token
  #   Note, the featurize function returns 2 Dataframes, the first has been filtered to remove insignficant tokens (below the idfThreshold)
  #         the second contains vectors with ALL tokens (we need these for accurate similarity functions during scoring later)
  #   Also Note, the full dataframe is not presently needed, so we discard it 
  _sigFeatures, junk = featurize(_tokens, idf_threshold=TCFG[4], sid="sourceId", tf=_termFreq)
  
  # We are tokenizing again because we also need a version here where we did NOT dedup all of our values.
  #   We need this so we can properly reconstruct all of the candidate pairs later.
  _tokensALL = tokenize(contextDictRaw.get(TCFG[0]), mode=TCFG[1], sid="sourceId").filter(size(col("tokens"))>0)
  
  # This time, all we care about is the DataFrame with all tokens
  # Similarly, the filtered version of this dataframe is not needed at this time, so we discard it 
  if TCFG[1]=="numeric":
    _allFeatures = numeric_featurize(_tokensALL, sid="sourceId")
  else:
    junk, _allFeatures = featurize(_tokensALL, idf_threshold=TCFG[4], sid="sourceId", tf=_termFreq)
  
  # Accumulate our results into featuresList
  featuresList.append({"name":TCFG[0], "tokenizer":TCFG[1], "fullFeatures":_allFeatures})
  
  # It is possible that a tokenizer is set up, but no binning is desired.
  # So, if a jaccardIndexThreshold has been set, let's assume user wants to do binning.
  if TCFG[2] is not None:
    _pairs = binning(_sigFeatures, threshold=TCFG[2], numHashes=TCFG[3]).drop("minHashJaccardDistance")
    #print(f"size of pairs for {TCFG[0]} is {_pairs.count()}")
    pairsList.append(_pairs)
    
    """
    We eliminated exact matches earlier and all we are getting from binning now are fuzzy matches.  
    In more other words... if:
      record1.text = "some cool words" 
      record2.text = "some cool words"
      record3.text = "some cooler words"
    Binning will generate one candidate pair between 2:3 (or 1:3 but not both).  We also need 1:2, which we are doing here.
    We self-joining source_to_feature to find exact matches within each context
    """
    thisContextData = source_to_feature.filter(col("contextName")==TCFG[0]).select("sourceId", "valueId")
    thisContextIdentPairs1 = thisContextData.withColumnRenamed("sourceId","sourceId1")
    thisContextIdentPairs2 = thisContextData.withColumnRenamed("sourceId","sourceId2")

    thisContextIdentPairs = (thisContextIdentPairs1.join(thisContextIdentPairs2, "valueId", "inner")
                             .filter(col("sourceId1")!=col("sourceId2"))
                             .drop("valueId")
                            )
    ident_pairs = ident_pairs.union(thisContextIdentPairs)
    #print(f"ident_pairs size after {TCFG[0]} is {ident_pairs.count()}")

"""Let's accumulate all of the candidate pairs from all of the binning here into one big dataframe
   We are still just looking at 2 columns: featureId1, featureId2
"""
all_pairs = reduce(DataFrame.unionAll, pairsList)

"""So, we have featureId1:featureId2, but we need to get back to sourceIds.  It will require 2 hops.
   Let's get to valueId1:valueId2 (remember source_to_feature maps all featureIds back to all of their original valueIds and sourceIds)
"""
all_pairs = (all_pairs
             .join(source_to_feature, all_pairs.featureId1==source_to_feature.featureId)
             .withColumnRenamed("valueId", "valueId1")
             .withColumnRenamed("contextName", "contextName1")
             .drop("sourceId", "featureId1", "featureId")
             .join(source_to_feature, all_pairs.featureId2==source_to_feature.featureId)
             .withColumnRenamed("valueId", "valueId2")
             .withColumnRenamed("contextName", "contextName2")
             .drop("sourceId", "featureId2", "featureId")
            )

"""Now we can join with source_to_feature again to get all sourceId::sourceId
   But we need to make sure we are always using matching Contexts.  
     e.g. an Address value and Address_street value could match, but we wouldn't want to generate a pair here
   Finally, after doing this series of joins, we should be back to our original record granularity, having undone all of the earlier
      optimization dedups.
"""
all_pairs = (all_pairs
             .join(
               source_to_feature,
               (all_pairs.valueId1==source_to_feature.valueId) & (all_pairs.contextName1==source_to_feature.contextName) 
             )
             .withColumnRenamed("sourceId", "sourceId1")
             .drop("valueId", "valueId1", "featureId", "contextName", "contextName1")
             .join(
               source_to_feature, 
               (all_pairs.valueId2==source_to_feature.valueId) & (all_pairs.contextName2==source_to_feature.contextName)
             )
             .withColumnRenamed("sourceId", "sourceId2")
             .drop("valueId", "valueId2", "featureId", "contextName", "contextName2")
            )

""" Combine all fuzzy pairs with all exact pairs """

all_pairs = all_pairs.union(ident_pairs)

"""Rearrange IDs so they are in deterministic (lexical order), for deduping
   If we don't do this step that it's possible we have ID1:ID2 AND ID2:ID1.  We need to sort them so that a dedup will catch them.
   This would be harder if all_pairs had more columns... but it's just 2 ID columns.  We'll join back in the actual features later.
""" 
all_pairs = (all_pairs
             .withColumn("sourceID1_temp", string_first(col("sourceId1"),col("sourceId2")))
             .withColumn("sourceID2_temp", string_last(col("sourceId1"),col("sourceId2")))
             .withColumn("sourceId1", col("sourceID1_temp"))
             .withColumn("sourceId2", col("sourceID2_temp"))
             .drop("sourceID1_temp","sourceID2_temp")
            ).dropDuplicates(["sourceId1", "sourceId2"]).filter(col("sourceId1")!=col("sourceId2"))

"""Drop pairs from within the same source unless selfDedup is configured
  If we are looking for matches across more than 1 dataset, we have the option of searching for dupes
    across AND within datasets, or just across.
  We have to loop over each source here because it can be configured differently for each source.
"""
for SRC in sources.collect():
  if not SRC[2]: #If selfDedup is False
    all_pairs = all_pairs.filter(~((col("sourceID1").contains(SRC[0])) & (col("sourceID2").contains(SRC[0]))))
    
all_pairs.write.format("delta").mode("overwrite").save(f"{intDataDir}/delta/SILVER/all_pairs")
all_pairs = spark.read.format("delta").load(f"{intDataDir}/delta/SILVER/all_pairs")

In [0]:
print(f"Total pairs to be considered: {all_pairs.count()} \nTotal Claims being considered: {all_pairs.select('sourceId1').distinct().count()} \nTotal PCR_Metas being considered: {all_pairs.select('sourceId2').distinct().count()}")

In [0]:
#################
"""SCORE PAIRS"""
#################

"""Join back in the original features so we can actually score the potential pairs"""
scored_pairs = all_pairs.select("sourceId1","sourceId2")
scoreColumnsList = []
rcols = []

# This loops over our actual context features
# The "fullFeatures" key:val pair within each element of featuresList has the schema: {sourceId, featureId, feature_vector}
# We can join on sourceID now and get the feature_vector for both sides of each candidate pair
# Then we can calculate jaccard and cosine similarities.
for featureInfo in featuresList:
  _thisName = featureInfo.get("name")
  _thisTokenizer = featureInfo.get("tokenizer")
  _thisFeature = featureInfo.get("fullFeatures").select("sourceId","features")
  scoreColumn = f"{_thisName}__{_thisTokenizer}"
  feature1 = f"{scoreColumn}__features1"
  feature2 = f"{scoreColumn}__features2"
  cosCol = f"{scoreColumn}__cosine_sim"
  numCol = f"{scoreColumn}_sim"
  
  #Note: There is a subtle thing happening here.
  #  First, and straightforwardly, we attach the feature vectors and use them to calculate the similarities.
  #  But afterward we do a groupBy on sourceId pairs again, so as to keep the max value of each similarity
  #  This is needed because the earlier joins could be cartesianing our pairs again.
  #  e.g. 
  #.    there may only be one candidate pair for ID1 and ID2,
  #     but ID1 could have two different feature vectors for the same context 
  #     (remember we may have mapped more than 1 column to a single context)
  #     so, maybe ID1 has two names - "Lucas" AND "Luke" - while ID2 only has one - "Luke"
  #     After this join we will again have 2 scored records for that one pair ID1:ID2:0% and ID1:ID2:100%.
  #     and, obviously, we keep the higher score.
  scored_pairs = (scored_pairs
                   .join(broadcast(_thisFeature), scored_pairs.sourceId1 == _thisFeature.sourceId, how="left")
                   .withColumnRenamed("features", feature1)
                   .select("sourceId1", "sourceId2", *rcols, feature1)
                   .join(broadcast(_thisFeature), scored_pairs.sourceId2 == _thisFeature.sourceId, how="left")
                   .withColumnRenamed("features", feature2)
                   .select("sourceId1", "sourceId2", *rcols, feature1, feature2)
                  )
  
  if _thisTokenizer == "numeric":
    scored_pairs = (scored_pairs
                    .withColumn(numCol, round(relative_numeric_sim(feature1, feature2), 3))
                    .groupBy("sourceId1","sourceId2")
                    .max(*rcols, numCol)
                   )
    rcols += [numCol]
                    
  else:
    scored_pairs = (scored_pairs
                    .withColumn(cosCol, round(cos_sim(col(feature1), col(feature2)), 3))
                    .groupBy("sourceId1","sourceId2")
                    .max(*rcols, cosCol)
                   )
    rcols += [cosCol]

  for _col in rcols:
    scored_pairs = scored_pairs.withColumnRenamed(f"Max({_col})", _col)
    
scored_pairs.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{intDataDir}/delta/GOLD/scored_pairs")
scored_pairs = spark.read.format("delta").load(f"{intDataDir}/delta/GOLD/scored_pairs")

In [0]:
# This list should contain all of our numeric independent variables to be used as features
keepers = [
  "FirstName__trigram__cosine_sim",
  "MiddleName__trigram__cosine_sim",
  "LastName__trigram__cosine_sim",
  "Address_street__trigram__cosine_sim",
  "Address_city__trigram__cosine_sim",
  "Address_county__trigram__cosine_sim",
  "Address_state__trigram__cosine_sim",
  "Address_zip__default__cosine_sim",
  "Gender__default__cosine_sim",
  "DOB__default__cosine_sim",
  "Age__numeric_sim",
  "Race__default__cosine_sim",
  "SSN__default__cosine_sim",
  "MBI__default__cosine_sim",
  "DriverLicNum__default__cosine_sim",
  "StateIssDriverLic__default__cosine_sim"
]

# For purpose of weighted scores, we only care about our similarities and IDs
for keeper in keepers:
  if keeper not in scored_pairs.columns:
    scored_pairs = scored_pairs.withColumn(keeper, lit(None))
weighted_input = scored_pairs.select(keepers + ["sourceID1", "sourceID2"])

MATCH_THRESHOLD = float(match_threshold)

predictions = (weighted_input
               .withColumn(
                 "pm_score", 
                 weighted_score(
                   col("FirstName__trigram__cosine_sim"),
                   col("MiddleName__trigram__cosine_sim"),
                   col("LastName__trigram__cosine_sim"),
                   col("Address_street__trigram__cosine_sim"),
                   col("Address_city__trigram__cosine_sim"),
                   col("Address_county__trigram__cosine_sim"),
                   col("Address_state__trigram__cosine_sim"),
                   col("Address_zip__default__cosine_sim"),
                   col("Gender__default__cosine_sim"),
                   col("DOB__default__cosine_sim"),
                   col("Age__numeric_sim"),
                   col("Race__default__cosine_sim"),
                   col("SSN__default__cosine_sim"),
                   col("MBI__default__cosine_sim"),
                   col("DriverLicNum__default__cosine_sim"),
                   col("StateIssDriverLic__default__cosine_sim")
                 )
               )
              ).withColumn("prediction", when(col("pm_score")>=MATCH_THRESHOLD, lit(1)).otherwise(lit(0)))
predictions.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{intDataDir}/delta/GOLD/predictions")
predictions=spark.read.format("delta").load(f"{intDataDir}/delta/GOLD/predictions")

In [0]:
# Is this where we should drop PM_SCORE < 85?

pm_thresh_below = predictions.filter(col("pm_score") < 85)
pm_above = predictions.filter(col("pm_score") >= 85)

# Might be too early to implement?
print(f"Total rows: {predictions.count()}") 
print(f"Total rows with PM score > 85: {pm_above.count()}")
print(f"Total rows with pm score < 85: {pm_thresh_below.count()}")
# Looks like we filter for > 85 Threshold when creating the graph frames below

In [0]:
'''Assign the appropriate patient ids to the claims from the PM predctions, using the below steps
1.) Relabel the appropriate identifying fields for CLM_UNIQ_ID and mp_id, so we can grab relevant information from claims_master and pcr_master tables respectively.
2.) Retrieve patient_ids and their respective dispatch_dates from pcr_master for the mp_ids in the claims-pcr_meta matches.
3.) Retrieve CLM_FROM_DT from claims_master table for the claims.
4.) Each claim-pcr_meta match has now been expanded to be a row for every patient_id associated with each mp_id in our matches.
5.) Evaluate each line as a 'match' if they have both a positive 'prediction' value AND the CLM_FROM_DT matches the PCR's dispatch date
'''
predictions=spark.read.format("delta").load(f"{intDataDir}/delta/GOLD/predictions")
window_spec = Window.partitionBy(col('sourceID1')).orderBy(col('pm_score').desc())
claim_predictions=predictions.withColumn("rank",rank().over(window_spec)).orderBy(col('sourceID1'),col('rank').asc())\
.select('sourceID1','sourceID2','pm_score','prediction','rank')\
.withColumnRenamed('sourceID1','CLM_UNIQ_ID')\
.withColumn('CLM_UNIQ_ID',regexp_extract(col('CLM_UNIQ_ID'), '(.)(__)(\w+)', 3))\
.withColumnRenamed('sourceID2','mp_id').withColumn('mp_id',upper(col('mp_id')))\
.withColumn('mp_id',regexp_extract(col('mp_id'), '(.)(__)(\w+)', 3))\
.join(spark.sql('''SELECT 
                    mp_id,
                    patient_id,
                    pcr_dispatch_date 
                    FROM (
                      SELECT mp_id,
                      patient_id,
                      cast(dispatch_timestamp as date) as pcr_dispatch_date, 
                      row_number() OVER (PARTITION BY mp_id,cast(dispatch_timestamp as date) ORDER BY patient_id DESC) rank 
                      FROM pcr_master
                       ) tmp 
                     WHERE rank <= 1'''), on="mp_id", how="inner")\
.join(spark.sql("select CLM_UNIQ_ID, CLM_FROM_DT from claims_master").withColumn('CLM_FROM_DT',to_date(col('CLM_FROM_DT'),'ddMMMyyyy')),on="CLM_UNIQ_ID",how="inner")\
.withColumn("match",when((col("prediction")==1) & (col("pcr_dispatch_date")==col("CLM_FROM_DT")), lit(1)).otherwise(lit(0)))
claim_predictions.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{intDataDir}/delta/GOLD/claim_predictions")

claim_predictions=spark.read.format("delta").load(f"{intDataDir}/delta/GOLD/claim_predictions")
claim_predictions.createOrReplaceTempView('Matches')
#This is for later validation and answers the questions: How many matches were there? How many matches also had the correct date? Of those without the correct date, what was the average days difference?
match=spark.sql('''select sum(case when max_match=1 then 1 else 0 end) as match_count,
              count(distinct case when max_pred=1 and max_match=0 then CLM_UNIQ_ID else null end) as pred_no_match_count,
              count(distinct case when max_pred=0 then CLM_UNIQ_ID else null end) as no_pred_no_match_count--,
              from (
                Select CLM_UNIQ_ID, 
                max(prediction) as max_pred,
                max(match) as max_match
                from Matches group by 1) a''').collect()
#these are claims that were thrown out at the minHash step as not having enough similarity with any PCRs to even be scored against them.
hash_misses = spark.sql('select count(CLM_UNIQ_ID) from claims_master where CLM_UNIQ_ID not in (select distinct CLM_UNIQ_ID from Matches)').collect()[0][0]
matched=match[0][0] #claims that had a PM score >=85 and a date match to at least one claim
nonMatch=match[0][1] #claims that had a PM score >=85 to at least one claim, but no date matches
nonPred=match[0][2]+hash_misses #claims that did not have any PM scores >= 85
print(f'There are {matched} matches \nThere are {nonMatch} non-matches\nThere are {nonPred} non-preds')

In [0]:
claim_predictions=spark.read.format("delta").load(f"{intDataDir}/delta/GOLD/claim_predictions")
claim_predictions.createOrReplaceTempView('Matches')
spark.sql('''
select
CLM_UNIQ_ID as Claim_ID,
mp_id,
pm_score,
pcr_dispatch_date,
clm_from_dt,
days_diff
from (
        select CLM_UNIQ_ID, mp_id,pm_score, pcr_dispatch_date, clm_from_dt,
        abs(datediff(clm_from_dt, pcr_dispatch_date)) as days_diff,
        rank() over (partition by clm_uniq_id order by pm_score desc, abs(datediff(clm_from_dt, pcr_dispatch_date)) asc, mp_id) as match_rank
        from matches 
        where clm_uniq_id in (
                              Select CLM_UNIQ_ID
                              from Matches 
                              group by 1
                              having max(prediction)=0
                              )
        ) as a
where match_rank = 1

Union

select CLM_UNIQ_ID,
'' as mp_id,
'' as pm_score,
'' as pcr_dispatch_date,
'' as clm_from_dt,
'' as days_diff
from claims_master 
where CLM_UNIQ_ID not in (select distinct CLM_UNIQ_ID from Matches)
''').display()

In [0]:
%sql
select 
CLM_UNIQ_ID as Claim_ID,
mp_id,
patient_id,
pm_score,
pcr_dispatch_date,
clm_from_dt,
days_diff
from (
 select CLM_UNIQ_ID, mp_id,pm_score, pcr_dispatch_date, clm_from_dt,patient_id,
        abs(datediff(clm_from_dt, pcr_dispatch_date)) as days_diff,
        rank() over (partition by clm_uniq_id order by abs(datediff(clm_from_dt, pcr_dispatch_date)) asc, pm_score desc, mp_id) as match_rank
from Matches
where CLM_UNIQ_ID in (Select CLM_UNIQ_ID
                      from Matches group by 1
                      having max(prediction)=1 and max(match)=0)
and pm_score >=85.0
) a
where match_rank = 1

In [0]:
%sql
--For claims that are non-matches, what is minimum days diff between service from date and dispatch date?
select CLM_UNIQ_ID, cast(min(abs(datediff(
      case when prediction=1 and match=0 then clm_from_dt else null end,
      case when prediction=1 and match=0 then pcr_dispatch_date else null end
  ))) as int) as min_days_diff 
from Matches
where CLM_UNIQ_ID in (Select CLM_UNIQ_ID
                      from Matches group by 1
                      having max(prediction)=1 and max(match)=0)
group by 1

In [0]:
#There was one case observed, where a claim was being matched to two nearly identical pcrs, but who happened to have different patient ids, the below picks one at random and drops the unneeded helper columns
spark.sql('select * from Matches').filter('match==1')\
.withColumn('window',rank().over(Window.partitionBy(col('CLM_UNIQ_ID')).orderBy(col('pm_score').desc(),col('patient_id').desc())))\
.filter('window==1').drop('rank','window','match','prediction')\
.createOrReplaceTempView('Matches')
print("There were {} claims matched to a PCR".format(spark.sql('Select * from matches').count()))
assert spark.sql('select CLM_UNIQ_ID, count(patient_id) from Matches group by 1 having count(patient_id)>1 order by CLM_UNIQ_ID'), "Claims matching to multiple PCRs..."

In [0]:
%sql
MERGE INTO claims_master a using(
                                select 
                                a.CLM_UNIQ_ID, a.CLM_TYPE_CD, a.Participant_NPI_1, a.Participant_NPI_source_1, a.Participant_NPI_2, a.Participant_NPI_source_2, a.CLM_FROM_DT, a.CLM_THRU_DT, a.geo_ptnt_pckp_sk,a.geo_drop_off_sk, a.CLM_FINL_ACTN_IND, a.DEMO_91_Flag, a.ET3_HCPC, a.HCPC_Modifier, a.CLM_SUBMSN_DT, a.BENE_MBI_ID, a.bene_BRTH_DT, a.BENE_LAST_NAME, a.BENE_1ST_NAME, a.BENE_MIDL_NAME,  a.BENE_LINE_1_ADR, a.BENE_LINE_2_ADR, a.BENE_LINE_3_ADR, a.BENE_LINE_4_ADR, a.BENE_LINE_5_ADR, a.BENE_LINE_6_ADR, a.SRC_USPS_STATE_CD, a.SRC_ZIP5_CD, a.BENE_SSN_NUM, a.BENE_SEX_CD, a.BENE_RACE_CD, b.patient_id
                                from claims_master a
                                left join Matches b
                                on a.CLM_UNIQ_ID=b.CLM_UNIQ_ID) b
  ON a.CLM_UNIQ_ID==b.CLM_UNIQ_ID
  WHEN MATCHED 
    THEN UPDATE SET *
  WHEN NOT MATCHED 
    THEN INSERT *

In [0]:
dbutils.fs.rm(f"{tempDataDir}/claims_to_rf", True)
spark.sql('''select CLM_UNIQ_ID, Participant_NPI_1, CLM_FROM_DT, CLM_THRU_DT, geo_ptnt_pckp_sk, ET3_HCPC, HCPC_Modifier, CLM_SUBMSN_DT, BENE_MBI_ID, patient_id from claims_master''')\
.withColumn('CLM_FROM_DT',regexp_replace(to_date(col('CLM_FROM_DT'),'ddMMMyyyy').cast('String'),'-',''))\
.withColumn('CLM_THRU_DT',regexp_replace(to_date(col('CLM_THRU_DT'),'ddMMMyyyy').cast('String'),'-',''))\
.withColumn('CLM_SUBMSN_DT',regexp_replace(to_date(col('CLM_SUBMSN_DT'),'ddMMMyyyy').cast('String'),'-',''))\
.withColumnRenamed('CLM_SUBMSN_DT','Submission')\
.withColumnRenamed('patient_id','patientid')\
.coalesce(1).write.mode('overwrite').format('csv').option('header', 'true').option('delimiter', '|').option('emptyValue','').save(f"{tempDataDir}/claims_to_rf")